In [1]:
from keras import Sequential
from keras.layers import Dense
from sklearn.model_selection import KFold

import pandas
import numpy as np
import statistics

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset = pandas.read_csv("data/table.csv", sep="\t")
dataset.head()

,relevant,depth,number_links,number_relevants,number_td,number_th,number_tr
0,1,8,8,19,10,0,5
1,1,8,8,18,10,0,5
2,1,8,8,17,10,0,5
3,1,8,8,16,10,0,5
4,1,8,8,15,10,0,5


In [3]:
# numpy dataset

data  = np.ndarray((len(dataset), 6), np.int)
label = np.ndarray((len(dataset), 1), np.int)

for i, row in dataset.iterrows():
    row = list(row)
    
    label[i]= row.pop(0)
    data[i] = np.fromiter(row, np.int)

In [4]:
def table_classifier():
    model = Sequential()
    model.add(Dense(32, input_shape=(6,), activation='tanh'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', 
              loss='mean_squared_error',
              metrics=['accuracy'])
    return model

In [5]:
# train & test
   |||||||||||||||||||||||||||
1) ||||||||||||||||||
                     |||||||||     OK
2)              ||||||||||||||
   ||||||||                        NI
    
    
    
loo = KFold(20)

res = []
for (i, (train_index, test_index)) in enumerate(loo.split(data)):
    print("{}/{}".format(i, loo.get_n_splits(data)))
    
    model = table_classifier()
    data_train,   data_test =  data[train_index],  data[test_index]
    label_train, label_test = label[train_index], label[test_index]
    
    model.fit(data_train, label_train, epochs=10, batch_size=8, shuffle=True)
    r = model.evaluate(data_test, label_test, verbose=False)
    res.append(r)

0/20
Epoch 1/10
1289/1289 [==============================] - 0s 214us/step - loss: 0.2106 - acc: 0.7199
Epoch 2/10
1289/1289 [==============================] - 0s 175us/step - loss: 0.1364 - acc: 0.8666
Epoch 3/10
1289/1289 [==============================] - 0s 174us/step - loss: 0.0979 - acc: 0.8984
Epoch 4/10
1289/1289 [==============================] - 0s 182us/step - loss: 0.0772 - acc: 0.9139
Epoch 5/10
1289/1289 [==============================] - 0s 171us/step - loss: 0.0682 - acc: 0.9162
Epoch 6/10
1289/1289 [==============================] - 0s 170us/step - loss: 0.0623 - acc: 0.9193
Epoch 7/10
1289/1289 [==============================] - 0s 158us/step - loss: 0.0580 - acc: 0.9271
Epoch 8/10
1289/1289 [==============================] - 0s 192us/step - loss: 0.0558 - acc: 0.9271
Epoch 9/10
1289/1289 [==============================] - 0s 188us/step - loss: 0.0531 - acc: 0.9286
Epoch 10/10
1289/1289 [==============================] - 0s 179us/step - loss: 0.0516 - acc: 0.9325
1/20

1289/1289 [==============================] - 0s 176us/step - loss: 0.0762 - acc: 0.9038
Epoch 6/10
1289/1289 [==============================] - 0s 185us/step - loss: 0.0693 - acc: 0.9061
Epoch 7/10
1289/1289 [==============================] - 0s 182us/step - loss: 0.0646 - acc: 0.9154
Epoch 8/10
1289/1289 [==============================] - 0s 192us/step - loss: 0.0622 - acc: 0.9232
Epoch 9/10
1289/1289 [==============================] - 0s 173us/step - loss: 0.0590 - acc: 0.9224
Epoch 10/10
1289/1289 [==============================] - 0s 160us/step - loss: 0.0570 - acc: 0.9247
17/20
Epoch 1/10
1290/1290 [==============================] - 1s 445us/step - loss: 0.2054 - acc: 0.7248
Epoch 2/10
1290/1290 [==============================] - 0s 171us/step - loss: 0.1230 - acc: 0.8744
Epoch 3/10
1290/1290 [==============================] - 0s 180us/step - loss: 0.0977 - acc: 0.8907
Epoch 4/10
1290/1290 [==============================] - 0s 168us/step - loss: 0.0842 - acc: 0.9008
Epoch 5/10
129

In [6]:
loss = list(map(lambda x: x[0], res))
acc = list(map(lambda x: x[1], res))
print(acc)

[0.8382352941176471, 1.0, 0.9705882352941176, 0.5882352941176471, 0.8823529411764706, 0.7058823529411765, 0.9705882352941176, 0.8529411764705882, 1.0, 0.9264705882352942, 0.9264705882352942, 1.0, 1.0, 0.9705882352941176, 0.5882352941176471, 1.0, 0.9264705882352942, 0.8955223880597015, 0.6567164183552585, 0.7761194029850746]


In [12]:
print("loss: ")
print("\t mean", statistics.mean(loss))
print("\t  var", statistics.variance(loss))

print("acc: ")
print("\t mean", statistics.mean(acc))
print("\t  var", statistics.variance(acc))

loss: 
	 mean 0.10005239165338783
	  var 0.00965919077229743
acc: 
	 mean 0.8737708516464723
	  var 0.019276900841263694


In [10]:
model.save("models/table_classifier.h5")